In [1]:
import os
from PIL import Image

In [19]:
X = []
Y = []
width = 50
height = 50
INPUT_CLASSES_DIR = 'face_age/'

import numpy as np

for folder_name,_,filenames in os.walk(INPUT_CLASSES_DIR):
    count = 0
    if folder_name !="face_age/":
        # print(folder_name, len(filenames))
        if int(folder_name[-3:]) < 75:
            for file in filenames:
                if count < 100:
                    file_path = folder_name +"/"+ file
                    image = Image.open(file_path).convert('L').resize((width, height))
                    X.append(np.array(image).flatten())
                    #X.append(np.array(image))
                    Y.append(int(folder_name[-3:]))
                    count += 1
    else:
        pass

In [3]:
import pandas as pd

lst = list(zip(X, Y))


df = pd.DataFrame(X)
df['age'] = Y
print(len(X))

# df = pd.DataFrame(lst, columns = ['image', 'age'])

df.head()



2194


,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,age
0,127,-115,-108,-103,-98,-94,-90,-87,-84,-78,...,125,-75,-63,-78,-88,-73,-55,-53,-46,1
1,-19,-50,-51,-46,-54,-58,-62,-63,-63,-60,...,-105,-112,-121,111,108,-128,88,103,-120,1
2,-56,121,77,80,-90,-54,-43,-41,-40,-41,...,68,84,85,94,101,104,110,117,118,1
3,-112,-109,-101,-96,-100,-109,-121,-112,-92,-104,...,0,0,0,0,0,0,0,0,0,1
4,86,-124,-67,-62,-55,-52,-53,-56,-59,-63,...,-55,-53,-46,-32,-34,-36,-46,-50,-42,1


In [4]:
Y.count(31)

30

In [5]:
# first = df.iloc[0]

# # each image is a 100x100 pixel 2D array where each pixel is an array of 3 values in form of RGB
# first['image'].shape

In [6]:
#df.to_csv('dataframe.csv')

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
import math


def plot_effect_of_C(data, hp1, hp2, hpv1, hpv2):

    means = []
    Cs = []

    for m, s, p in data:
        if p[hp1] == hpv1 and p[hp2] == hpv2:
            means.append(m)
            Cs.append(p['C'])

    plt.plot(range(len(Cs)),means)
    plt.xticks(range(len(Cs)), Cs)
    plt.title('hyperparameter vs accuracy')
    plt.xlabel('hyperparameter C')
    plt.ylabel('accuracy')
    plt.show()

def train_test_SVM(features, target):
    # X_train, X_test, y_train, y_test = train_test_split(features, target)
    
    # kernel = ['poly', 'rbf', 'sigmoid']
    # c_values = [50, 10, 1.0, 0.1, 0.01]
    # gamma = ['scale', 'auto']

    # grid = dict(kernel=kernel,C=c_values,gamma=gamma)
    # cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # grid_search = GridSearchCV(estimator=SVC(), param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)

    # grid_result = grid_search.fit(X_train, y_train)
    
    # print("Best params", grid_result.best_params_)
    # means = grid_result.cv_results_['mean_test_score']
    # stds = grid_result.cv_results_['std_test_score']
    # params = grid_result.cv_results_['params']

    # for mean, stdev, param in zip(means, stds, params):
    #     print("%f (%f) with: %r" % (mean, stdev, param))


    # plot_effect_of_C(zip(means, stds, params), 'gamma', 'kernel', grid_result.best_params_['gamma'], grid_result.best_params_['kernel'])

    # print(target.head())
    # print(features.head())
        
    data_rows = []
    total_score = 0
    iters = 1
    for i in range(iters):
        X_train, X_test, y_train, y_test = train_test_split(features, target)

        model = SVC(max_iter=10000, C = 1.2, kernel = 'rbf').fit(X_train, y_train)

        actual = model.predict(X_test)
        expected = y_test.array
        correct = 0

        dictScores = { '0-5': 0, '6-10': 0, '11-15': 0, '16-25': 0, '+25': 0}
        for index, age in enumerate(actual):
            if abs(age - expected[index]) < 10:
                correct += 1

            diff = abs(age - expected[index])

            if diff <= 5:
                dictScores['0-5']+=1
            elif diff <= 10:
                dictScores['6-10']+=1
            elif diff <= 15:
                dictScores['11-15']+=1
            elif diff <= 25:
                dictScores['16-25']+=1
            else:
                dictScores['+25']+=1

            # print(age, expected[index])
        
        # print(correct, len(X_test))

        train_acc = model.score(X_train, y_train)
        test_acc = model.score(X_test, y_test)

        total_score += test_acc

        data_rows.append({
            "train_acc": train_acc,
            "test_acc": test_acc,
            "our_test_acc": correct/len(X_test)
        })

        a = model.predict([X_test.iloc[500]])
        e = expected[500]
        print(a)
        print(e) 

        print(dictScores)

    test_df = pd.DataFrame(data_rows)
    print('average score', total_score/iters, '\n',test_df)

   

In [8]:
target = df['age']
df2 = df.drop(columns=['age'])
features = df2

In [9]:
train_test_SVM(features, target)

[7]
4
{'0-5': 136, '6-10': 92, '11-15': 61, '16-25': 110, '+25': 150}
average score 0.0273224043715847 
    train_acc  test_acc  our_test_acc
0   0.881459  0.027322      0.378871


In [46]:

def train_test_logistic_regressor(features, target):
    # X_train, X_test, y_train, y_test = train_test_split(features, target)
    
    # # solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    # # penalty =  ['l1','l2']
    # c_values = [10, 1.0, 0.1, 0.01]


    # grid = dict(C=c_values)
    # cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)

    # grid_result = grid_search.fit(X_train, y_train)
  
    # print("Best params", grid_result.best_params_)
    # means = grid_result.cv_results_['mean_test_score']
    # stds = grid_result.cv_results_['std_test_score']
    # params = grid_result.cv_results_['params']

    # for mean, stdev, param in zip(means, stds, params):
    #     print("%f (%f) with: %r" % (mean, stdev, param))

    # plot_effect_of_C(zip(means, stds, params), 'solver', 'penalty', grid_result.best_params_['solver'], grid_result.best_params_['penalty'])


    data_rows = []
    total_score = 0
    iters = 1
    for i in range(iters):
        X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=.6)
        model = LogisticRegression(max_iter=10000, C = 1.5).fit(X_train, y_train)

        actual = model.predict(X_test)
        expected = y_test.array
        correct = 0

        dictScores = { '0-5': 0, '6-10': 0, '11-15': 0, '16-25': 0, '+25': 0}
        for index, age in enumerate(actual):
            if abs(age - expected[index]) < 10:
                correct += 1

            diff = abs(age - expected[index])

            if diff <= 5:
                dictScores['0-5']+=1
            elif diff <= 10:
                dictScores['6-10']+=1
            elif diff <= 15:
                dictScores['11-15']+=1
            elif diff <= 25:
                dictScores['16-25']+=1
            else:
                dictScores['+25']+=1
                
        train_acc = model.score(X_train, y_train)
        test_acc = model.score(X_test, y_test)

        total_score += test_acc

        print(dictScores)


        data_rows.append({
            "train_acc": train_acc,
            "test_acc": test_acc,
            "our_test_acc": correct/len(X_test)
        })

    test_df = pd.DataFrame(data_rows)
    print('average score', total_score/iters, '\n',test_df)

In [47]:
train_test_logistic_regressor(features, target)

{'0-5': 181, '6-10': 133, '11-15': 115, '16-25': 150, '+25': 299}
average score 0.02164009111617312 
    train_acc  test_acc  our_test_acc
0    0.99848   0.02164      0.333713


In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, target)

model = SVC(max_iter=10000).fit(X_train, y_train)

print(X_train.head())


      0    1    2    3    4    5    6    7    8    9    ...  390  391  392  \
1872   17   15   15   31   58   80   99  116 -124 -115  ... -125 -126  119   
2151  -71  -58  -47  -46  -29  -29  -30  -21  -23  -22  ...  -71  -76  -94   
67     71  118 -114 -105  -99  -92  -87  -87  -85  -86  ...  -68  -56  -50   
1586   33   20   20   20   16   22   37   51   53   51  ...   84   81   70   
66    -68  -55  -54  -52  -51  -38  -38  -40  -41  -44  ...  -74  -87 -120   

      393  394  395  396  397  398  399  
1872   76   60   59   75   82   77   77  
2151 -113 -117 -103  -66  -61  -62  -61  
67    -68  -68  -67  -67  -67  -68  -72  
1586   61   57   53   48   51   50   51  
66    114  122  127  123  121  120  120  

[5 rows x 400 columns]


In [13]:
# print(y_train)

In [14]:
i = []
file_path = "./zab.png"
image = Image.open(file_path).convert('L').resize((width, height))
i.append(np.array(image).flatten())


# file_path = "./face_age/035/2283.png"
# image = Image.open(file_path).convert('L').resize((width, height))
# i.append(np.array(image).flatten())


df_zabib = pd.DataFrame(i)
# print(df_zabib.iloc[0])

print(model.predict([df_zabib.iloc[0]]))

print(image.show())

[9]
None
